In [ ]:
#import dependencies
from bs4 import BeautifulSoup
import requests
import re
import numpy as np
import pandas as pd

In [ ]:
# build url for scraping
base_url = 'https://www.basketball-reference.com/draft/'

# seasons to scrape
seasons = np.arange(2000,2020)

# create arrays to hold data
names = []
ids = []

In [ ]:
# iterate thru list of seasons
for season in seasons:
    
    # build url
    url_end = f'NBA_{season}.html'
    url = base_url + url_end
    
    # retrieve page with the requests module
    response = requests.get(url)
    
    # create BeautifulSoup object; parse with 'lxml'
    soup = BeautifulSoup(response.text, 'lxml')
    
    # get table from page
    draft_table = soup.find('table', id='stats')
    
    # get rows with player information
    rows = draft_table.findAll('td', {'data-stat' : 'player'}, class_= 'left')
    
    # get player ids from rows
    for row in rows:        
        
        # pull player name
        try:
            player_name = row.find('a').text
            names.append(player_name)
        except:
            print('Player has no id. Moving to next player...')
        
        # pull player id
        try:
            player_href = row.find('a')['href']
            player_formatted = player_href.replace('/', '.')
            player_id = player_formatted.split('.')[3]
            ids.append(player_id)
        except:
            None

In [ ]:
# base url for scraping
base_url = 'https://www.basketball-reference.com/players/'

# empty array to hold data
cbb_ids = []

In [ ]:
for player_id in ids:
    
    # build final url
    url_end = f'{player_id[0]}/{player_id}.html'
    url = base_url + url_end
    
    # retrieve page with the requests module
    response = requests.get(url)

    # create BeautifulSoup object; parse with 'lxml'
    soup = BeautifulSoup(response.text, 'lxml')

    # get relevant html
    results = soup.find_all('div', id='inner_nav')
    
    for result in results:
        # parse CBB id
        try:
            page_link = result.find('a', text='College Basketball at Sports-Reference.com')['href']
            formatted = page_link.replace('/', '.')
            cbb_ids.append(formatted.split('.')[7])
            print(f'Match found for {player_id}!')
        except:
            print(f'No match found for {player_id}.')
            cbb_ids.append(None)

In [ ]:
# build id mapping dataframe
id_map = pd.DataFrame({'player_name': names,
                       'pro_id': ids,
                       'cbb_id': cbb_ids})

# export to .csv
id_map.to_csv('data/id_map.csv', index = False)